In [1]:
import os
from PIL import Image
import cv2
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

# model = keras.models.load_model("Mobilefacenet-TF2-coral_tpu/model/model_with_mask_clf.h5")
model = keras.models.load_model("Mobilefacenet-TF2-coral_tpu/pretrained_model/training_model/inference_model_0.993.h5")
resize = tf.keras.layers.Resizing(
    112,
    96,
    interpolation='bilinear',
)
model.summary()

2022-08-23 19:04:55.061891: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-23 19:04:55.098143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-23 19:04:55.098398: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-23 19:04:55.102701: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 112, 96, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 114, 98, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 56, 48, 64)   1728        ['zero_padding2d[0][0]']         
                                                                                                  
 batch_normalization (BatchNorm  (None, 56, 48, 64)  256         ['conv2d[0][0]']             

In [2]:
def preprocess(img):
    img = cv2.resize(img, (96, 112))
    size = img.shape
    # print(size)
    img = resize(tf.convert_to_tensor(img))
    img = (img - 127.5) / 128.0
    img = np.expand_dims(img, axis=0)
    return img




path = "dataset/lfw_funneled"

img_names = os.listdir(path)


In [3]:
all_img_path = []

for i in os.listdir(path):
    if i[-3:] == "txt":
        continue
    for j in os.listdir(path+'/'+i):
        if "ipynb_checkpoints" in j:
            continue
        all_img_path.append([i,j])


In [4]:
import matplotlib.pyplot as plt

if "embeddings_mobilefacenet" not in os.listdir():
    os.mkdir("embeddings_mobilefacenet")

def generate_embeds(path, model):
    img = preprocess(plt.imread(path))

    with tf.device('/device:GPU:0'):
        return model(tf.convert_to_tensor(img))

In [6]:
from tqdm.auto import tqdm
import gc

gc.enable()

for x, path_ in tqdm(enumerate(all_img_path), total = len(all_img_path)):
    i = generate_embeds(path+"/"+'/'.join(path_), model).numpy()[0]
    
    if path_[0] not in os.listdir(f"embeddings_mobilefacenet"):
        os.mkdir(f"embeddings_mobilefacenet/{path_[0]}")
    
    np.save(f"embeddings_mobilefacenet/{path_[0]}/{path_[1]}.npy", i)

  0%|          | 0/13137 [00:00<?, ?it/s]

2022-08-23 19:05:55.459196: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-08-23 19:05:56.560187: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-23 19:05:56.561015: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-23 19:05:56.561298: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-08-23 19:05:56.561756: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-23 19:05:56.561791: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-08-23 19:05:57.597083: I tensorflow/stream_executor/cuda/c

In [7]:
def euclidean_dist(x1, x2):
    return np.linalg.norm(x1 - x2)


In [11]:
euclidean_dist(np.load("embeddings_mobilefacenet/Aaron_Peirsol/Aaron_Peirsol_0001.jpg.npy"), np.load("embeddings_mobilefacenet/Aaron_Peirsol/Aaron_Peirsol_0002.jpg.npy"))

27.58307

In [12]:
euclidean_dist(np.load("embeddings_mobilefacenet/Aaron_Peirsol/Aaron_Peirsol_0001.jpg.npy"), np.load("embeddings_mobilefacenet/Aaron_Patterson/Aaron_Patterson_0001.jpg.npy"))

59.934143